In [42]:
import pandas as pd
import numpy as np
import cv2

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split 
from tensorflow.keras.utils import to_categorical
import tensorflow as tf
from tensorflow import keras

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.models import Sequential

In [43]:
to_train = pd.read_csv("/Users/elsa/Desktop/Bootcamp The Bridge/The_Bridge/Ds_Bootcamp_Elsa/Ejercicios/competicion_kaggle/Competicion_how_am_I_feeling/how-am-i-feeling/train_set.csv")


In [8]:
to_train

,label,id_img,path
0,happy,22373,happy/22373.jpg
1,happy,21433,happy/21433.jpg
2,happy,12418,happy/12418.jpg
3,happy,21278,happy/21278.jpg
4,happy,8081,happy/08081.jpg
...,...,...,...
6171,sadness,11346,sadness/11346.jpg
6172,sadness,4441,sadness/04441.jpg
6173,sadness,15236,sadness/15236.jpg
6174,sadness,27361,sadness/27361.jpg


In [50]:
# Convertimos todas las imagenes en un array:
X_train = []
for elem in to_train.path:
    path = "totrain/totrain/" + elem
    X_train.append(cv2.imread(path))

X_train = np.array(X_train)
#print(X_train)
print("####################")
print(X_train.shape)

####################
(6176, 48, 48, 3)


In [58]:

def grayscale(data, dtype='float32'):
    # luma coding weighted average in video systems
    r, g, b = np.asarray(.3, dtype=dtype), np.asarray(.59, dtype=dtype), np.asarray(.11, dtype=dtype)
    rst = r * data[:, :, :, 0] + g * data[:, :, :, 1] + b * data[:, :, :, 2]
    # add channel dimension
    rst = np.expand_dims(rst, axis=3)
    return rst

train_images_grey = grayscale(X_train)
train_images_grey.shape

(6176, 48, 48, 1)

In [60]:
train_images_grey = train_images_grey / 255.0
train_images_grey.shape


(6176, 48, 48, 1)

In [61]:
# Necesito obtener los labels para poder conseguir y_train
lista_label = []
for elem in to_train.label:
    #lista_label.append(cv2.imread(elem))
    lista_label.append(elem)

# tomo una posición para que no me imprima toda la lista
lista_label[6]

'happy'

In [62]:
# 2._ OTRA FORMA DE APLICAR LABELENCODER
y_train = np.asarray(lista_label)
y_train

array(['happy', 'happy', 'happy', ..., 'sadness', 'sadness', 'sadness'],
      dtype='<U7')

In [63]:
# APLICAR LABELENCODER a y_train
le = LabelEncoder()
y_train = le.fit_transform(y_train)
# train_labels = y_train
y_train

array([0, 0, 0, ..., 1, 1, 1])

In [64]:
# 4-Creamos sets de Entrenamiento y Test, Validación y Preprocesar

train_X,test_X,train_Y,test_Y = train_test_split(train_images_grey,y_train,test_size=0.2)
print('Training data shape : ', train_X.shape, train_Y.shape)
print('Testing data shape : ', test_X.shape, test_Y.shape)

Training data shape :  (4940, 48, 48, 1) (4940,)
Testing data shape :  (1236, 48, 48, 1) (1236,)


In [121]:
tf.config.run_functions_eagerly(True)

INPUT_SHAPE = (48,48,1)

model = keras.models.Sequential()

# 1 capa convolutiva con 8 neuronas & 1 MaxPool quedando las dimensiones de la imagen a la mitad
model.add(keras.layers.Conv2D(filters=8, kernel_size=(3, 3), strides=(1, 1), activation="relu", padding="valid", input_shape=INPUT_SHAPE))
model.add(keras.layers.MaxPool2D(pool_size=(2, 2)))

#1 dropout 0.25
model.add(keras.layers.Dropout(rate=0.25))

#1 Flatten
model.add(keras.layers.Flatten())

#1 dense con 32 neuronas
model.add(keras.layers.Dense(units=32, activation="relu"))

# 1 dense con 10 (salida)
model.add(keras.layers.Dense(units=10, activation="softmax"))
model.summary()

Model: "sequential_19"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_13 (Conv2D)           (None, 46, 46, 8)         80        
_________________________________________________________________
max_pooling2d_13 (MaxPooling (None, 23, 23, 8)         0         
_________________________________________________________________
dropout_20 (Dropout)         (None, 23, 23, 8)         0         
_________________________________________________________________
flatten_14 (Flatten)         (None, 4232)              0         
_________________________________________________________________
dense_41 (Dense)             (None, 32)                135456    
_________________________________________________________________
dense_42 (Dense)             (None, 10)                330       
Total params: 135,866
Trainable params: 135,866
Non-trainable params: 0
_______________________________________________

In [122]:
#Estos son los mejores optimizadores para este modeloo: adam, Ftrl, nadam
model.compile(optimizer="Ftrl",
              loss='mse',
              metrics=['accuracy'])

In [123]:
history = model.fit(train_images_grey, y_train, epochs=10)

Epoch 1/10
193/193 [==============================] - 5s 25ms/step - loss: 0.3845 - accuracy: 0.5311
Epoch 2/10
193/193 [==============================] - 5s 25ms/step - loss: 0.3845 - accuracy: 0.5319
Epoch 3/10
193/193 [==============================] - 5s 27ms/step - loss: 0.3845 - accuracy: 0.5319
Epoch 4/10
193/193 [==============================] - 5s 26ms/step - loss: 0.3845 - accuracy: 0.5319
Epoch 5/10
193/193 [==============================] - 5s 26ms/step - loss: 0.3845 - accuracy: 0.5319
Epoch 6/10
193/193 [==============================] - 5s 27ms/step - loss: 0.3845 - accuracy: 0.5319
Epoch 7/10
193/193 [==============================] - 5s 26ms/step - loss: 0.3845 - accuracy: 0.5319
Epoch 8/10
193/193 [==============================] - 5s 27ms/step - loss: 0.3845 - accuracy: 0.5319
Epoch 9/10
193/193 [==============================] - 5s 28ms/step - loss: 0.3845 - accuracy: 0.5319
Epoch 10/10
193/193 [==============================] - 5s 24ms/step - loss: 0.3845 - accura

In [110]:
# Evaluar Exactitud
test_loss, test_acc = model.evaluate(train_images_grey,  y_train, verbose=2)

print('\nTest accuracy:', test_acc). ### ME HA SIDO IMPOSIBLE CONSEGUIR MAS ACCURACY

193/193 - 2s - loss: 0.3845 - accuracy: 0.0000e+00

Test accuracy: 0.0


## Aplicamos to_pred & sample submission

In [124]:
to_test = pd.read_csv("/Users/elsa/Desktop/Bootcamp The Bridge/The_Bridge/Ds_Bootcamp_Elsa/Ejercicios/competicion_kaggle/Competicion_how_am_I_feeling/how-am-i-feeling/test_set.csv"). # to_test = to_pred

In [125]:
to_test

,id_img,path
0,18341,to_pred/18341.jpg
1,13176,to_pred/13176.jpg
2,23945,to_pred/23945.jpg
3,15968,to_pred/15968.jpg
4,18382,to_pred/18382.jpg
...,...,...
4112,8966,to_pred/08966.jpg
4113,12111,to_pred/12111.jpg
4114,16629,to_pred/16629.jpg
4115,24322,to_pred/24322.jpg


#### Aplico todos los pasos que he necesitado para X_train

In [126]:
to_predd = []
for i,elem in enumerate(to_pred):
    to_predd.append(np.argmax(to_pred[i]))
len(to_predd)

4117

In [159]:
# Convertimos todas las imagenes en un array:
to_pred = []
for elem in to_test.path:
    path = "to_pred/to_pred/" + elem
    to_pred.append(np.argmax(path))

to_pred = np.array(to_pred)
print(to_pred)
print("####################")
print(to_pred.shape)
len(to_pred)

[0 0 0 ... 0 0 0]
####################
(4117,)


4117

In [154]:
to_pred = to_test.path
to_pred = np.array(to_pred)
to_pred.shape

(4117,)

In [ ]:
# NO CONSIGO SALIR DE ESTE PASO

In [177]:
test_images_grey = grayscale(to_pred)
test_images_grey.shape

IndexError: too many indices for array

In [ ]:
test_images_grey = test_images_grey / 255.0

In [ ]:
predictions_submit = model.predict(to_pred)
predictions_submit

In [ ]:
submission = pd.DataFrame({"id_img": to_pred["0"], "label": predictions_submit})
submission

In [132]:
sample = pd.read_csv("sample_submission.csv")
sample

,id_img,label
0,18341,sadness
1,13176,sadness
2,23945,happy
3,15968,happy
4,18382,happy
...,...,...
4112,8966,sadness
4113,12111,sadness
4114,16629,sadness
4115,24322,sadness


In [133]:
sample.shape

(4117, 2)

In [ ]:
if submission.shape == sample.shape:
    if submission.columns.all() == sample.columns.all():
        if submission.id.all() == sample.id.all():
            print("you're ready to submit!")
            submission.to_csv("to_submit.csv", index = False)
            # ¡¡¡¡¡¡¡ADD INDEX = FALSE!!!!!!!!!